In [1]:
#Following NewYork libraries 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.4                |           py36_0         877 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         961 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The following packages will be UPDATED:

  conda                                        4.6.3-py36_0 --> 4.6.4-py36_0
  geopy              conda-forge/linux-64::geopy-1.11.0-py~ --> conda-forge/noarch::geopy-1.18.1-py_0



geopy-1.18.1         

In [2]:
#Load and explore the data that has been established 
Combined_table = pd.read_csv("maintable.csv")
Combined_table.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Combined_table['Borough'].unique()),
       Combined_table.shape[0]
    )
)

The dataframe has 10 boroughs and 102 neighborhoods.


In [4]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [5]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Combined_table['Latitude'], Combined_table['Longitude'], Combined_table['Borough'], Combined_table['Neighborhood']):
    label = '{},{}' .format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
map_Toronto

In [6]:
#Utilize the Foursquare API to explore the neighborhoods and segment them
#Define Foursquare Credentials and Version

CLIENT_ID = 'OYHCJKKQ45CABI4SQCNEPXW3HP55BKLQLY0HCOHPRIKHBURB' # your Foursquare ID
CLIENT_SECRET = '5YGV2U3HIIABFYE41PUQULS44OCKRZH0GNWUVTE22HWBQXSJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OYHCJKKQ45CABI4SQCNEPXW3HP55BKLQLY0HCOHPRIKHBURB
CLIENT_SECRET:5YGV2U3HIIABFYE41PUQULS44OCKRZH0GNWUVTE22HWBQXSJ


In [7]:
#However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods Scarborough Borough!let's slice the original dataframe and create a new dataframe of the Manhattan data.
scarborough_data = Combined_table[Combined_table['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
#Let's get the geographical coordinates of Scarborough.
address = 'Scarborough,Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [10]:
# create map of Scarborough using latitude and longitude values
map_scarborough_data = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Borough'], scarborough_data['Neighborhood']):
    label = '{},{}' .format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough_data)  
map_scarborough_data

In [12]:
#Utilize the Foursquare API to explore the neighborhoods and segment them
#Define Foursquare Credentials and Version

CLIENT_ID = 'OYHCJKKQ45CABI4SQCNEPXW3HP55BKLQLY0HCOHPRIKHBURB' # your Foursquare ID
CLIENT_SECRET = '5YGV2U3HIIABFYE41PUQULS44OCKRZH0GNWUVTE22HWBQXSJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OYHCJKKQ45CABI4SQCNEPXW3HP55BKLQLY0HCOHPRIKHBURB
CLIENT_SECRET:5YGV2U3HIIABFYE41PUQULS44OCKRZH0GNWUVTE22HWBQXSJ


In [13]:
#Let's explore the first neighborhood in our dataframe.
scarborough_data.loc[0,'Neighborhood']

'Malvern, Rouge'

In [14]:
#Get the neighborhood's latitude and longitude values.
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

print('Latitude and longitude values are {}, {}.'.format(neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values are 43.8066863, -79.19435340000003.


Now, let's get the top 100 venues that are in Malvern, Rouge within a radius of 800 meters.

In [15]:
#let's get the top 100 venues in Malvern, Rouge within a radius of 800 meters. First, let's create the GET request URL. Name your URL url.
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 800 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OYHCJKKQ45CABI4SQCNEPXW3HP55BKLQLY0HCOHPRIKHBURB&client_secret=5YGV2U3HIIABFYE41PUQULS44OCKRZH0GNWUVTE22HWBQXSJ&v=20180605&ll=43.8066863,-79.19435340000003&radius=800&limit=100'

In [16]:
#Send the GET request and examine the resutls
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c67f864351e3d13abcd3183'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 43.81388630720001,
    'lng': -79.18439528916868},
   'sw': {'lat': 43.7994862928, 'lng': -79.20431151083137}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80228301948931,
          'lng': -79.19856472801668}],
        'dis

In [17]:
#From the Foursquare lab in the previous module, we know that all the information is in the items key. #Before we proceed, let's borrow the get_category_type function from the Foursquare lab.
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
# we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Wendy's,Fast Food Restaurant,43.807448,-79.199056
3,Harvey's,Fast Food Restaurant,43.800106,-79.198258
4,Tim Hortons,Coffee Shop,43.802000,-79.198169


In [19]:
#Let´s see how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

12 venues were returned by Foursquare.


Explore the neighbourhood in SCARBOROUGH 

In [21]:
#explore the neighbourhood in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
#Now write the code to run the above function on each neighborhood and create a new dataframe
Scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Highland Creek, Port Union, Rouge Hill
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


In [23]:
#Let's check the size of the resulting dataframe
print(Scarborough_venues.shape)
Scarborough_venues.head()

(91, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [24]:
#Let's check how many venues were returned for each neighborhood
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",9,9,9,9,9,9
"Cliffcrest, Cliffside, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre, Wexford Heights",8,8,8,8,8,8
"East Birchmount Park, Ionview, Kennedy Park",9,9,9,9,9,9


In [25]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


#Analyze each Neigbhourhood

In [26]:
#Analyze each neigbhourhood
# one hot encoding
Scar_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scar_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scar_onehot.columns[-1]] + list(Scar_onehot.columns[:-2])
Scar_onehot = Scar_onehot[fixed_columns]

Scar_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Food Truck,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Port Union, Rouge Hill",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
#let's examine the new dataframe size
Scar_onehot.shape

(91, 55)

In [28]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Scar_grouped = Scar_onehot.groupby('Neighborhood').mean().reset_index()
Scar_grouped

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Food Truck,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station
0,Agincourt,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.250000,0.000,0.0,0.000000,0.00,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.25,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.250000,0.0,0.0,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.5,0.000,0.000000,0.000000,0.5,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.25,0.000000,0.000000,0.00,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.250000,0.0,0.0,0.000000,0.000000,0.000000
3,Cedarbrae,0.000,0.000000,0.142857,0.000,0.142857,0.142857,0.0,0.000000,0.000,0.0,0.000000,0.00,0.142857,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.142857,0.00,0.000000,0.142857,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.142857,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.000,0.000000,0.000000,0.000,0.200000,0.000000,0.0,0.000000,0.000,0.2,0.100000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.1,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.1,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.1,0.000000,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.00,0.000000,0.111111,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.111111,0.0,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.111111,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.111111,0.0,0.000,0.111111,0.222222,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.111111,0.000000,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.000,0.333333,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.333333,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.333333,0.0,0.0,0.000000,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.125,0.0,0.000000,0.00,0.000000,0.125000,

In [29]:
#Let's confirm the new size
Scar_grouped.shape

(16, 55)

In [30]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in Scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scar_grouped[Scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0  Clothing Store  0.25
1    Skating Rink  0.25
2  Breakfast Spot  0.25
3          Lounge  0.25
4       Pet Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0                Park   0.5
1          Playground   0.5
2   Accessories Store   0.0
3        Intersection   0.0
4  Italian Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0  General Entertainment  0.25
1           Skating Rink  0.25
2                   Café  0.25
3        College Stadium  0.25
4      Accessories Store  0.00


----Cedarbrae----
                 venue  freq
0     Hakka Restaurant  0.14
1  Fried Chicken Joint  0.14
2   Athletics & Sports  0.14
3               Bakery  0.14
4                 Bank  0.14


----Clairlea, Golden Mile, Oakridge----
                  venue  freq
0                Bakery   0.2
1              Bus Line   0.2
2          Intersection   0.1
3          Soccer Field

In [31]:
#Let's put that into a pandas dataframe. First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scar_grouped['Neighborhood']

for ind in np.arange(Scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Train Station,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Train Station,Caribbean Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff, Cliffside West",Café,Skating Rink,College Stadium,General Entertainment,Auto Garage,Clothing Store,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Clothing Store,Food Truck,Fast Food Restaurant
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Food Truck,Soccer Field,Fast Food Restaurant,Intersection,Bus Station,Park,Train Station,Clothing Store
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Noodle House,Thai Restaurant,Fast Food Restaurant,Italian Restaurant,Pharmacy,Fried Chicken Joint,Chinese Restaurant,Caribbean Restaurant,Electronics Store
6,"Cliffcrest, Cliffside, Scarborough Village West",Motel,American Restaurant,Skating Rink,Train Station,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Thrift / Vintage Store,Brewery,Latin American Restaurant,Pet Store,Chinese Restaurant,Train Station,Clothing Store,Food Truck,Fast Food Restaurant
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Train Station,Bus Station,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Hobby Shop,Breakfast Spot,Auto Garage
9,"Guildwood, Morningside, West Hill",Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Train Station,Chinese Restaurant,Food Truck,Fast Food Restaurant


In [33]:
#Cluster Neighborhoods. Let us run k-means to cluster the neighborhood into 5 clusters.

# import k-means from clustering stage
from sklearn.cluster import KMeans

scar_data = scarborough_data.drop(15)

# set number of clusters

kclusters = 5

Scar_grouped_clustering = Scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
Scar_merge = scar_data

# add clustering labels
Scar_merge['Cluster Labels'] = kmeans.labels_

# merge data to add latitude/longitude for each neighborhood
Scar_merge = Scar_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scar_merge.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Train Station,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Electronics Store,Discount Store,Department Store,Convenience Store
1,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497,4,Bar,Train Station,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Train Station,Chinese Restaurant,Food Truck,Fast Food Restaurant
3,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Train Station,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
4,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Clothing Store,Food Truck,Fast Food Restaurant


In [35]:
#Finally, let's visualize the resulting clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scar_merge['Latitude'], Scar_merge['Longitude'], Scar_merge['Neighborhood'], Scar_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster. 

In [36]:
#CLUSTER 1
Scar_merge.loc[Scar_merge['Cluster Labels'] == 0, Scar_merge.columns[[1] + list(range(5, Scar_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",Fast Food Restaurant,Train Station,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Electronics Store,Discount Store,Department Store,Convenience Store
2,"Guildwood, Morningside, West Hill",Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Train Station,Chinese Restaurant,Food Truck,Fast Food Restaurant
3,Woburn,Coffee Shop,Korean Restaurant,Train Station,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
4,Cedarbrae,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Clothing Store,Food Truck,Fast Food Restaurant
5,Scarborough Village,Smoke Shop,Playground,Train Station,Caribbean Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Train Station,Bus Station,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Hobby Shop,Breakfast Spot,Auto Garage
7,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Food Truck,Soccer Field,Fast Food Restaurant,Intersection,Bus Station,Park,Train Station,Clothing Store
8,"Cliffcrest, Cliffside, Scarborough Village West",Motel,American Restaurant,Skating Rink,Train Station,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,"Birch Cliff, Cliffside West",Café,Skating Rink,College Stadium,General Entertainment,Auto Garage,Clothing Store,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
11,"Maryvale, Wexford",Middle Eastern Restaurant,Bakery,Breakfast Spot,Sandwich Place,Shopping Mall,Accessories Store,Auto Garage,Fast Food Restaurant,Chinese Restaurant,Discount Store


In [37]:
#CLUSTER 2
Scar_merge.loc[Scar_merge['Cluster Labels'] == 1, Scar_merge.columns[[1] + list(range(5, Scar_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Thrift / Vintage Store,Brewery,Latin American Restaurant,Pet Store,Chinese Restaurant,Train Station,Clothing Store,Food Truck,Fast Food Restaurant


In [38]:
#CLUSTER 3
Scar_merge.loc[Scar_merge['Cluster Labels'] == 2, Scar_merge.columns[[1] + list(range(5, Scar_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Train Station,Chinese Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store


In [39]:
#CLUSTER 4
Scar_merge.loc[Scar_merge['Cluster Labels'] == 3, Scar_merge.columns[[1] + list(range(5, Scar_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Upper Rouge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
#CLUSTER 5
Scar_merge.loc[Scar_merge['Cluster Labels'] == 4, Scar_merge.columns[[1] + list(range(5, Scar_merge.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Port Union, Rouge Hill",Bar,Train Station,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
14,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Train Station,Caribbean Restaurant,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
